In [1]:
import pronouncing
from itertools import product
import functools
import operator
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from tqdm import tqdm_notebook as tqdm
import string
import gensim

In [2]:
model = gensim.models.KeyedVectors.load_word2vec_format('InferSent/dataset/fastText/crawl-300d-2M.vec')  

In [15]:
def remove_punctuation(phrase): return phrase.translate(str.maketrans({a:None for a in string.punctuation}))

In [4]:
stop = set(stopwords.words('english'))

INCLUDE_STOP = True
def synonyms(word):
    if not INCLUDE_STOP and word in stop:
        return [word]
    
    most_similar =model.most_similar(positive=[word], topn=50)
    
    return list(set([word[0].lower() for word in most_similar if len(pronouncing.phones_for_word(word[0])) > 0]))[:50]

synonyms('slow')

/home/matthew/.local/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


['slows',
 'sluggish',
 'slowest',
 'slowness',
 'slowing',
 'rapid',
 'slowed',
 'steady',
 'slow',
 'plodding',
 'fast',
 'slower']

In [25]:
def get_meter(phrase):
    phrase = remove_punctuation(phrase) 
    syllables = 0 
    stresses = []
    for word in word_tokenize(phrase):
        phonemes = pronouncing.phones_for_word(word)[0]
        syllables += pronouncing.syllable_count(phonemes)
        stresses.append(list(map(int, pronouncing.stresses(phonemes))))

    return syllables, stresses

In [26]:
get_meter('that was amazing')

(5, [[1], [1], [0, 1, 0]])

In [37]:
def check_meter(line, meter, n_syllables=None): # meter is an array with False for unstressed, True for stressed
    cycle_len = len(meter)
    meter_step = 0
    
    syllables, stresses = get_meter(line)
    
    if n_syllables and syllables is not n_syllables:
        return False
    
    expected_next = False # not stressed
    
    for stress in stresses:
        if len(stress) == 1:
            meter_step += 1
            continue # one syllable can be stressed or not stressed
        
        for syllable in stress:
            if syllable == 2:
                meter_step += 1
                continue # lightly stressed, can be either
                
            if (syllable == 1) == meter[meter_step % cycle_len]: 
                meter_step += 1
                continue
            else:
                return False
            
    if meter_step % cycle_len == 0: # end at the correct step
        return True
    else:
        return False

In [44]:
def check_iambic(line):
    return check_meter(line, [False, True])

def check_trochaic(line):
    return check_meter(line, [True, False])

def check_dactylic_hexameter(line):
    return check_meter(line, [True, False, False] * 5 + [True, False])

In [45]:
print(check_iambic('How do I love thee? Let me count the ways.'))
print(check_iambic('Is this smart enough to catch bad meter?'))

print(check_dactylic_hexameter('This is the forest primeval, the murmuring pines and the hemlock'))

True
False
True


In [30]:
test_poem = \
'''How do I love thee? Let me count the ways.
I love thee to the depth and breadth and height
My soul can reach, when feeling out of sight
For the ends of being and ideal grace.
I love thee to the level of every day's
Most quiet need, by sun and candle-light.
I love thee freely, as men strive for right.
I love thee purely, as they turn from praise.
I love thee with the passion put to use
In my old griefs, and with my childhood's faith.
I love thee with a love I seemed to lose
With my lost saints. I love thee with the breath,
Smiles, tears, of all my life; and, if God choose,
I shall but love thee better after death.
'''.split('\n')
for line in test_poem:
    try:
        if not check_iambic(line):
            print(line)
    except:
        pass

For the ends of being and ideal grace.
I love thee to the level of every day's


In [ ]:
# WIP
def make_chunk_iambic(words):
    word_synonyms = list(map(synonyms, words))
    
    iambic = []
    # https://stackoverflow.com/questions/32074543/how-to-get-the-length-of-an-itertools-product
    n_possible = functools.reduce(operator.mul, map(len, word_synonyms), 1)
    for possible in tqdm(product(*word_synonyms), total=n_possible):
        potential = ' '.join(possible)
        try:
            if check_iambic(potential):
                return potential

        except:
            continue

In [20]:
def make_iambic(line, return_early=10):
    words = line.split(' ') # word_tokenize(line)
    word_synonyms = list(map(synonyms, words))
    
    for i, syns in enumerate(word_synonyms):
        if len(syns) == 0:
            word_synonyms[i] = [words[i]]
            
    iambic = []
    # https://stackoverflow.com/questions/32074543/how-to-get-the-length-of-an-itertools-product
    n_possible = functools.reduce(operator.mul, map(len, word_synonyms), 1)
    for possible in tqdm(product(*word_synonyms), total=n_possible):
        potential = ' '.join(possible)
        if check_iambic(potential):
            iambic.append(potential)
            
            if return_early and len(iambic) >= return_early:
                return iambic
    return iambic

In [21]:
INCLUDE_STOP = False
iambic = make_iambic("hot sauce contained in a nice tin can may be a little weird")

Error: some words have unknown pronunciation
Getting synonyms...


/home/matthew/.local/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


HBox(children=(IntProgress(value=0, max=4), HTML(value='')))

In [13]:
corpus = word_tokenize(open('sermon_mount_NIV.txt').read())

In [ ]:
def find_iambic(corpus):
    

In [ ]:
[0, 0, 1]